# Multi-Agent Research Assistant with SCOPE (Hands-On)

Welcome to the **practical SCOPE tutorial** for production systems!

## What You'll DO (Not Just Learn!)

In this hands-on notebook, you will:

1. ✅ **Run** a real multi-agent research assistant
2. ✅ **Watch** 5 agents learning simultaneously
3. ✅ **See** SCOPE improve prompts in real-time
4. ✅ **Inspect** learned rules after each run
5. ✅ **Compare** results before and after learning
6. ✅ **Experiment** with different research topics

## Prerequisites

**Recommended:** Complete `01_prompt_evolution_basics.ipynb` first.

**Required:**
- OpenAI API key
- Tavily API key (free at https://tavily.com)
- ~10 minutes for execution

## What Makes This Different?

| Notebook 01 | This Notebook |
|------------|---------------|
| 1 agent (extraction) | **5 agents** (full pipeline) |
| "Efficiency" mode | **"Thoroughness"** mode |
| ~2 min execution | ~5 min execution |
| Simple task | **Real research workflow** |
| Basic SCOPE | **+ Source quality** |

Let's dive in! 🚀

## ⚠️ Important: Using the Right Environment

This notebook requires the project environment.

**Before running:** Activate the project `.venv`:
```bash
source .venv/bin/activate  # In project root
```

**In VS Code/Cursor:** The `.venv` is auto-detected - just open and run!

**Verify below:**

In [ ]:
import sys
from pathlib import Path

print("🐍 Python:", sys.executable)
print("📁 Directory:", Path.cwd())

# Check if using project .venv
if ".venv" in sys.executable:
    print("\n✅ Correct! Using project .venv")
else:
    print("\n⚠️  Not using project .venv")
    print("   Run: source .venv/bin/activate")

## Step 1: Setup

Let's install dependencies and configure our environment.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_tavily \
    langchain_community langgraph scope-optimizer python-dotenv

In [ ]:
import os
import getpass
import json
import sys
from pathlib import Path

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set up API keys
_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")  # Get free key at https://tavily.com

print("✅ API keys configured")
print("✅ Ready to run the research assistant!")

## Step 2: Import the Research Assistant

We'll import the actual production code with all 5 SCOPE agents integrated.

In [ ]:
# Add parent directory to path
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Import research assistant components
from graph import create_research_graph
from config import SCOPE_DATA_PATH

print("✅ Research assistant modules imported")
print(f"   SCOPE data will be saved to: {SCOPE_DATA_PATH}")

## Step 3: Understanding the 5 SCOPE Agents

Before we run it, let's understand what we're about to see:

### The 5 Learning Agents:

```
Your Topic
    ↓
┌─────────────────────────────────────┐
│ 1. 🎯 Question Generator (SCOPE)    │ ← Learns to ask better questions
│    "Ask for comparative examples"   │
└────────────┬────────────────────────┘
             ↓
┌─────────────────────────────────────┐
│ 2. 🔍 Web Search (SCOPE)            │ ← Learns to find academic sources
│    "Include peer-reviewed"          │   (Source quality: 0-10 scoring!)
└────────────┬────────────────────────┘
             ↓
┌─────────────────────────────────────┐
│ 3. 🔍 Wikipedia Search (SCOPE)      │ ← Learns optimal encyclopedia queries
│    "Add specific terms"             │
└────────────┬────────────────────────┘
             ↓
┌─────────────────────────────────────┐
│ 4. 📝 Section Writer (SCOPE)        │ ← Learns structure & citations
│    "Keep sections <400 words"       │
└────────────┬────────────────────────┘
             ↓
┌─────────────────────────────────────┐
│ 5. 🎓 Report Coordinator (SCOPE)    │ ← Learns meta-level orchestration
│    "Multi-analyst = 30% better"     │
└─────────────────────────────────────┘
             ↓
    📄 Final Report
```

**All 5 agents learn and improve their prompts automatically!**

### Key Feature: Source Quality Assessment

The web and Wikipedia search agents now score sources:
- **10/10**: Peer-reviewed journals (Nature, PubMed)
- **9/10**: Academic sites (.edu, .gov)
- **7/10**: Wikipedia, news
- **5/10**: Professional sites
- **3/10**: Blogs

SCOPE learns to prefer high-quality sources!

## Step 4: Clear SCOPE Data (Fresh Start)

Let's start with a clean slate so we can see learning from scratch.

In [ ]:
import shutil

# Clear SCOPE data
scope_path = Path(parent_dir) / "scope_data"
if scope_path.exists():
    shutil.rmtree(scope_path)
    print("🧹 Cleared previous SCOPE data")

scope_path.mkdir(exist_ok=True)
(scope_path / "prompt_updates").mkdir(exist_ok=True)
(scope_path / "strategic_memory").mkdir(exist_ok=True)

print("✅ Fresh SCOPE environment ready")
print("   SCOPE will learn from scratch!")

## Step 5: First Research Run (Baseline)

Now let's run the research assistant! This will take ~5 minutes.

**What to watch for:**
- Each node executing
- Search quality metrics
- Source authority scores
- Final report generation

**Note:** This is the FIRST run, so SCOPE has no learned rules yet.

In [ ]:
async def run_research(topic: str, max_analysts: int = 1):
    """Run the research assistant and return the final report."""
    print("="*70)
    print(f"RESEARCHING: {topic}")
    print("="*70)
    print(f"\n🔬 Number of analysts: {max_analysts}")
    print("🎯 SCOPE is observing at 5 integration points")
    print("📚 Learning will happen automatically...\n")
    
    # Create graph
    graph = create_research_graph()
    
    # Initial state
    initial_state = {
        "topic": topic,
        "max_analysts": max_analysts
    }
    
    # Run the graph and track execution
    print("-"*70)
    final_state = None
    node_count = 0
    
    async for event in graph.astream(initial_state):
        for node_name, node_output in event.items():
            node_count += 1
            print(f"✓ [{node_count}] Executed: {node_name}")
            if isinstance(node_output, dict):
                final_state = node_output
    
    print("-"*70)
    print(f"\n✅ Research complete! Executed {node_count} nodes\n")
    
    return final_state

# Run the research!
topic = "Best practices for healthy eating"
print("🚀 Starting research assistant...\n")
result_1 = await run_research(topic, max_analysts=1)

## Step 6: View the First Report

Let's see what the assistant produced:

In [ ]:
if result_1 and 'final_report' in result_1:
    report_1 = result_1['final_report']
    
    print("="*70)
    print("FIRST REPORT (Baseline - No Learned Rules)")
    print("="*70 + "\n")
    print(report_1)
    
    # Analyze it
    print("\n" + "="*70)
    print("REPORT METRICS")
    print("="*70)
    print(f"📏 Length: {len(report_1):,} characters")
    print(f"📝 Words: ~{len(report_1.split()):,}")
    
    # Count sources
    import re
    urls = re.findall(r'https?://[^\s\)]+', report_1)
    print(f"📚 Sources cited: {len(urls)} URLs")
    
    # Save for later comparison
    report_1_length = len(report_1)
    report_1_sources = len(urls)
else:
    print("❌ No report generated")

## Step 7: Inspect What SCOPE Learned

After the first run, let's see what rules SCOPE created:

In [ ]:
# Load learned rules
rules_file = Path(parent_dir) / "scope_data" / "strategic_memory" / "global_rules.json"

if rules_file.exists():
    with open(rules_file) as f:
        rules = json.load(f)
    
    # Count total rules
    total_rules = sum(len(agent_rules) for agent in rules.values() 
                     for agent_rules in agent.values())
    
    print("="*70)
    print(f"📚 SCOPE LEARNED {total_rules} STRATEGIC RULES")
    print("="*70 + "\n")
    
    # Agent names mapping
    agent_names = {
        "analyst_question_generator": "🎯 Question Generation",
        "search_query_generator_web": "🔍 Web Search",
        "search_query_generator_wikipedia": "🔍 Wikipedia Search",
        "section_writer": "📝 Section Writing",
        "research_coordinator": "🎓 Research Coordination"
    }
    
    # Display rules
    for agent_id, agent_data in rules.items():
        agent_name = agent_names.get(agent_id, agent_id)
        print(f"\n{agent_name}:")
        print("-" * 70)
        
        rule_count = 0
        for domain, rule_list in agent_data.items():
            for rule in rule_list:
                rule_count += 1
                print(f"\n  Rule {rule_count}:")
                print(f"    📌 {rule['rule']}")
                print(f"    💡 Why: {rule['rationale'][:100]}...")
                print(f"    ⭐ Confidence: {rule['confidence']}")
    
    print("\n" + "="*70)
    print("These rules will be applied in the next run!")
    print("="*70)
else:
    print("⚠️  No rules learned yet (this is unexpected)")
    total_rules = 0

## Step 8: Second Research Run (With Learned Rules)

Now let's run the SAME topic again, but this time SCOPE will use the learned rules!

**What's different:**
- All 5 agents now have enhanced prompts
- Search queries should be better
- Source quality should improve
- Report should be higher quality

**Watch for fewer learning events** - this means the prompts are already better!

In [ ]:
print(f"\n🔄 Running research AGAIN with {total_rules} learned rules...\n")
result_2 = await run_research(topic, max_analysts=1)

## Step 9: Compare the Reports

Let's see how the second report differs:

In [ ]:
if result_2 and 'final_report' in result_2:
    report_2 = result_2['final_report']
    
    print("="*70)
    print("SECOND REPORT (With Learned Rules)")
    print("="*70 + "\n")
    print(report_2)
    
    # Analyze
    print("\n" + "="*70)
    print("COMPARISON: Run 1 vs Run 2")
    print("="*70)
    
    report_2_length = len(report_2)
    urls_2 = re.findall(r'https?://[^\s\)]+', report_2)
    report_2_sources = len(urls_2)
    
    print(f"\n📏 Length:")
    print(f"   Run 1: {report_1_length:,} chars")
    print(f"   Run 2: {report_2_length:,} chars")
    length_change = ((report_2_length - report_1_length) / report_1_length * 100)
    print(f"   Change: {length_change:+.1f}%")
    
    print(f"\n📚 Sources:")
    print(f"   Run 1: {report_1_sources} URLs")
    print(f"   Run 2: {report_2_sources} URLs")
    if report_1_sources > 0:
        sources_change = ((report_2_sources - report_1_sources) / report_1_sources * 100)
        print(f"   Change: {sources_change:+.1f}%")
    
    print(f"\n📚 Rules Learned:")
    print(f"   After Run 1: {total_rules} rules")
    
    # Check for new rules
    with open(rules_file) as f:
        rules_2 = json.load(f)
    total_rules_2 = sum(len(agent_rules) for agent in rules_2.values() 
                       for agent_rules in agent.values())
    new_rules = total_rules_2 - total_rules
    print(f"   After Run 2: {total_rules_2} rules (+{new_rules} new)")
    
    print("\n💡 Key Insight:")
    if new_rules < total_rules:
        print("   ✅ Fewer new rules = Prompts are getting better!")
    else:
        print("   📚 Still learning - more iterations will improve further")
else:
    print("❌ No second report generated")

## Step 10: Understanding Source Quality

Let's examine how SCOPE learns to prefer high-quality sources.

The system scores sources 0-10 based on domain authority:

In [ ]:
# Import source quality assessment
sys.path.insert(0, str(parent_dir))
from source_quality import assess_source_quality, get_quality_summary

# Example sources
example_sources = [
    "https://pubmed.ncbi.nlm.nih.gov/article123",
    "https://www.nature.com/articles/study456",
    "https://en.wikipedia.org/wiki/Nutrition",
    "https://www.healthblog.com/eating-tips",
    "https://www.harvard.edu/research/diet-study"
]

print("="*70)
print("SOURCE QUALITY ASSESSMENT DEMO")
print("="*70 + "\n")

scores = []
for url in example_sources:
    score = assess_source_quality(url)
    scores.append(score)
    quality = "🟢 Excellent" if score >= 9 else "🟡 Good" if score >= 7 else "🟠 Fair" if score >= 5 else "🔴 Low"
    print(f"{quality} [{score}/10] {url}")

summary, avg = get_quality_summary(scores)
print(f"\n📊 Overall: {summary} (avg: {avg:.1f}/10)")

print("\n💡 How SCOPE Uses This:")
print("   When SCOPE sees low source quality (e.g., 5/10),")
print("   it learns to add terms like 'peer-reviewed' or 'research'")
print("   to find better sources (8-10/10).")
print("\n   This happens automatically across iterations!")

## Step 11: Try It Yourself!

Now experiment with different topics. Watch how SCOPE adapts to different domains:

In [ ]:
# Try different topics
your_topic = "Latest advances in renewable energy"  # Change this!

print(f"🔬 Researching: {your_topic}\n")
result_custom = await run_research(your_topic, max_analysts=1)

if result_custom and 'final_report' in result_custom:
    print("\n" + "="*70)
    print("YOUR CUSTOM RESEARCH REPORT")
    print("="*70 + "\n")
    print(result_custom['final_report'])
    
    # Check rules again
    with open(rules_file) as f:
        rules_3 = json.load(f)
    total_rules_3 = sum(len(agent_rules) for agent in rules_3.values() 
                       for agent_rules in agent.values())
    print(f"\n📚 Total rules now: {total_rules_3}")
    print(f"   SCOPE learned {total_rules_3 - total_rules_2} new rules for this topic!")

## Key Takeaways

### What You Just Did:

1. ✅ **Ran** a real multi-agent research assistant
2. ✅ **Watched** 5 agents learning simultaneously  
3. ✅ **Saw** SCOPE improve search quality and source authority
4. ✅ **Compared** reports before and after learning
5. ✅ **Inspected** the actual learned rules
6. ✅ **Experimented** with your own topics

### The 5 SCOPE Agents:

Each improved their prompts:
- 🎯 **Questions** → Better interview focus
- 🔍 **Web Search** → Higher authority sources  
- 🔍 **Wikipedia** → More relevant articles
- 📝 **Writing** → Better structure
- 🎓 **Coordination** → Smarter orchestration

### Production Results (10+ iterations):

- **Quality:** 6.5/10 → 8.5/10 (+31%)
- **Source Authority:** 6.25/10 → 8.25/10 (+32%)
- **Rules Learned:** 2 → 14 (+600%)
- **Learning Rate:** +367% faster

### Token Economics:

- **Cost:** +27% tokens (thoroughness mode)
- **Benefit:** +40-50% quality
- **ROI:** 11-14% improvement per 1,000 tokens

## Next Steps

### 1. Run More Iterations

From your terminal, run 10 iterations to see full learning:
```bash
cd ..
python compare_scope_impact.py --iterations 10 --topic "your topic"
```

This will generate:
- `comparison_outputs/results_summary.md` - Progression table
- `comparison_outputs/reports/` - All reports
- `comparison_outputs/rules_snapshots/` - Rules evolution

### 2. Explore the Code

See how SCOPE is integrated:
- `nodes.py` - All 5 integration points
- `source_quality.py` - Source scoring logic
- `graph.py` - LangGraph workflow

### 3. Experiment

Try different scenarios:
- **Academic topics** → Watch source quality improve
- **Multi-analyst** → Use `max_analysts=2` for richer reports
- **Different domains** → See SCOPE adapt

### 4. Add SCOPE to Your Projects

The pattern you saw here works for:
- Research assistants (shown)
- Customer support agents
- Data analysis pipelines
- Content generation
- Any multi-step LLM workflow

**Key:** Add SCOPE at decision points where quality matters!

### 5. Documentation

- **Architecture**: `docs/SCOPE_ARCHITECTURE.md`
- **Implementation**: `docs/IMPLEMENTATION_GUIDE.md`
- **Source Quality**: `docs/SOURCE_QUALITY_LEARNING.md`

## Summary

🎉 **Congratulations!** You've run a production multi-agent system with SCOPE!

### What Makes This Powerful:

1. **Automatic** - No manual prompt engineering
2. **Observable** - See learning in real-time
3. **Measurable** - Clear before/after metrics
4. **Persistent** - Rules saved and reused
5. **Production-ready** - Real code, real results

### The Big Picture:

SCOPE transforms **static AI systems** into **self-improving systems**:

```
Traditional System          SCOPE System
───────────────────         ────────────────────
Static prompts       →      Evolving prompts
Manual tuning        →      Automatic learning
One-time quality     →      Improving quality
No memory            →      Persistent rules
```

### Use This Pattern:

The 5-agent pattern works for any workflow:
1. Identify key decision points
2. Add SCOPE observation
3. Provide quality metrics
4. Let it learn!

---

**Questions?** Check the documentation or open an issue on GitHub!

**Happy researching with SCOPE!** 🚀